In [5]:
#TRain a transformer

# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt 


--2025-01-21 21:35:01--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  3.20MB/s    in 0.3s    

2025-01-21 21:35:01 (3.20 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()


In [3]:
print(f"length of the dataset in characters: {len(text)}")

length of the dataset in characters: 1115394


In [4]:
##Sort the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
all_chars = ''.join(chars)

print(f"{all_chars}")
print(f"{vocab_size} chars")
      


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65 chars


Tokenizer

In [5]:
#Lets work on tokenization

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decoder = lambda l: [itos[c] for c in l]



In [ ]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print("Every integer in data represents a single character ", data.shape)


Every integer in data represents a single character  torch.Size([1115394])


In [7]:
#split
n = int(0.9*len(data))

train_data = data[:n]
val_data = data[n:]

In [8]:
context_length = 8
train_data[:context_length+1]

"8 examples come out of nine characters given the context for interpreting the next prediction "

'8 examples come out of nine characters given the context for interpreting the next prediction '

In [9]:
x = train_data[:context_length]
y = train_data[1:context_length+1]

for t in range(context_length):
    context = x[:t+1]
    target = y[t]
    
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
torch.manual_seed(1337)
batch_size = 4
context_length = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_length, (batch_size,)) #tensor of four integers representing four characters from the text
    # So for the random characters, we get i and then up to the block size. Then we put in a bigger tensor
    x = torch.stack([data[i:i+context_length] for i in ix])
    #same thing for y
    y = torch.stack([data[i+1:i+context_length+1] for i in ix])
    
    return x,y

xb, yb = get_batch('train')

print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

#This is where we show all of the 'examples' within our dataset. This is helpful for incorporating different context lengths
for b in range(batch_size):
    for t in range(context_length):
        context = xb[b, :t+1]
        target = yb[b,t]
        
        print(f"when input is {context.tolist()} the target: {target}")



inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [31]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        
        #Create embedding table where each index(representing individual characters) 
        # will pull out a row from the embedding table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
        
    def forward(self, idx, targets = None):
       
        #idx and targets are both (b, context_length) tensor of integers
        logits = self.token_embedding_table(idx) #output is (B,T,C)
        
        if targets is None: 
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C) #-> for cross entropy
            targets = targets.view(B*T) #reshaping
            
            #negative log likelihood
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    
    def generate(self, idx, max_new_tokens):
        
        #idx is (B,context_length) array of indices in the current context. Predicting token by token
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            
            #focus only on the last example (where we consider the max token length TODO: Understand why here)
            #These are the predictions for what comes next
            logits = logits[:,-1,:]
            
            #Apply softmax to get the probabilities
            probs = F.softmax(logits, dim = -1) #B, C
            
            idx_next = torch.multinomial(probs, num_samples = 1) #for each context length in the batch return the next selection based on probabilities distribution
            
            idx = torch.cat((idx, idx_next), dim = 1) #concatenate to the idx to create (B, T+1)
            #print(f"current status of idx: {idx}")
            
        return idx
    

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

starting_index = torch.zeros((1,1), dtype = torch.long) #Kicking off generation with zero tensors
sample_generated = m.generate(starting_index, max_new_tokens=100)
print(f"sample generated: {sample_generated.shape}")
#Sample generated works on the order of batches so we need to grab the return batch (it's effectively nested in a list)
batch_of_interest = sample_generated[0]

#Turn it into a list and decode
decoded = decoder(batch_of_interest.tolist())
"""(batch size, context_length (really number fo tokens we are looking at, 
embedding length per token))"""

print(f"Decoded output: {''.join(decoded)}")

sample generated: torch.Size([1, 101])
Decoded output: 
Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [ ]:
#Create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [39]:


batch_size = 32
for steps in range(1000):
    
    #get batch
    xb, yb = get_batch('train')
    
    #forward pass
    logits, loss = m(xb, yb)
    
    #zeroing the gradients
    optimizer.zero_grad(set_to_none = True)
    #getting gradients for parameters
    loss.backward()
    #upgrade parameters using optimizer
    optimizer.step()
    
print(loss.item())
    


2.6088967323303223


In [41]:
starting_index = torch.zeros((1,1), dtype = torch.long) #Kicking off generation with zero tensors
sample_generated = m.generate(starting_index, max_new_tokens=300)
print(f"sample generated: {sample_generated.shape}")
#Sample generated works on the order of batches so we need to grab the return batch (it's effectively nested in a list)
batch_of_interest = sample_generated[0]

#Turn it into a list and decode
decoded = decoder(batch_of_interest.tolist())
"""(batch size, context_length (really number fo tokens we are looking at, 
embedding length per token))"""

print(f"Decoded output: {''.join(decoded)}")

sample generated: torch.Size([1, 301])
Decoded output: 
AchinkOvin Hooly zolIn kthitollvkefa; 'vezIO$ARO:
HO:
By tu
PQ-d?e n ick:isanfeConus: tas, bycon,
Eli!

IHALETETEOUy-l, sthe.
CONEDuse Mlitiard tf'd EnQhOMyarerytrevNTa!
atD: tanTo sis; hitRWhedwid.
puDYonymy thandoTowQ!
ve:$ys s clllcUzPUpithinEneshh f r Isptlve II'A:'I!
pJOFtomukeg.
G wf tYZxrolb

